In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv() #.env 파일의 환경변수를 불러옴
api_key = os.getenv("OPENAI_API_KEY") # 환경변수에서 키 가져오기
client = OpenAI (api_key=api_key)

In [ ]:
file_obj = client.files.create(
file=open("data.csv", "rb"),
purpose="assistants"
)
file_id = file_obj.id
print("Uploaded file ID:", file_id)
assistant_ci = client.beta.assistants.create(
name="DataAnalyzer",
instructions="당신은 데이터 분석을 돕는 어시스턴트입니다. Python 코드를 작성하여 사용자의 요청을 해결하세요.",
model="gpt-4-1106-preview", # GPT-4 모델 사용 (코드 인터프리터 활용에 적합)
tools=[{"type": "code_interpreter"}] #코드 인터프리터 툴 활성화
)


In [ ]:
#3. 새로운 대화 스레드 시작 (코드 인터프리터 어시스턴트용)
thread_ci = client.beta.threads.create()
#4. 사용자 메시지 생성 (파일 첨부하여 질문)
question = "첨부된 파일의 모든 숫자를 더하면 얼마인가요?"
user_msg = client.beta.threads.messages.create(
thread_id=thread_ci.id,
role="user",
content=question,
attachments=[{
}]
)
"file_id": file_id,
"tools":[{"type": "code_interpreter"}] #문자열 대신 객체 형태로 변경
#5. 어시스턴트 실행 (코드 인터프리터 사용)
run_ci = client.beta.threads.runs.create_and_pol1(
thread_id=thread_ci.id,
assistant_id=assistant_ci.id
)
#6. 완료 후 어시스턴트의 답변 확인
messages_ci = client.beta.threads.messages.list(thread_id=thread_ci.id)
assistant_reply = messages_ci.data[0] # 첫 번째 메시지가 어시스턴트의 최종 응답
print(f"Assistant: {assistant_reply.content[0].text.value}")